In [ ]:
from selenium import webdriver

# pip install pandas in command line
import pandas as pd

# import packages to set wait time for page to load before running Selenium code
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

# packages to download images
import urllib
import urllib.request

# regex
import re

# sorting
import string

In [ ]:
letters = string.ascii_uppercase
df = pd.DataFrame(columns = ['name', 'uni', 'title', 'department'])

username = input("Username:")
password = input("Password:")

driver = webdriver.Chrome()

for i in letters:
    driver.get("https://directory.columbia.edu/people/browse/students;jsessionid=AE18155A2A30B9D6D4AEE0653247E6A4?filter.initialLetter=" + str(i))

    try:
        driver.find_element_by_id("username").send_keys(username)
        driver.find_element_by_id("password").send_keys(password)
        driver.find_element_by_xpath('//*[@id="fm1"]/div[3]/input[4]').send_keys(Keys.RETURN)

    except:
        pass

    current_url = driver.current_url
    ending = "filter.initialLetter=" + str(i)
    print(ending)
    print(len(df.name))
    
    try:
        for i in range(0,150):
            new_ending = 'page=' + str(i)

            new_url = re.sub(ending, new_ending, current_url)
            #print(new_url)

            driver.get(new_url)

            for i in range(2,42):
                s = []
                try:
                    name = driver.find_element_by_xpath('/html/body/div/a/form/table/tbody/tr[5]/td[2]/div[1]/table/tbody/tr[' + str(i) + ']/td[1]/a')
                    s.append(name.text)
                
                    try:
                        uni = name.get_attribute('href')
                        s.append(uni)
                        
                        try:
                            department = driver.find_element_by_xpath('/html/body/div/a/form/table/tbody/tr[5]/td[2]/div[1]/table/tbody/tr[' + str(i) + ']/td[2]/div[2]')
                            s.append(department.text)

                            try:
                                title = driver.find_element_by_xpath('/html/body/div/a/form/table/tbody/tr[5]/td[2]/div[1]/table/tbody/tr[' + str(i) + ']/td[2]/div[1]')
                                s.append(title.text)
                            
                            except:
                                s.append("None")
                        except:
                            pass
                    except:
                        pass
                except:
                    pass
                
                df.loc[len(df)] = s
    except:
        pass

driver.close()

In [ ]:
# clean uni column
df['uni'] = df['uni'].str[-5:]

In [ ]:
# drop title column
df.drop('title', axis =1, inplace=True)

In [ ]:
# clean department column
df['department'] = df['department'].str[7:]

In [ ]:
# save full directory
df.to_csv('CU_directory.csv')

In [ ]:
# only CBS
df_CBS = df[df['department'] == "Student, GRADUATE SCHOOL OF BUSINESS"]

In [ ]:
# reset index
df_CBS.reset_index(inplace= True)
df_CBS.drop('index', inplace = True, axis =1)

In [ ]:
# drop department
df_CBS.drop('department', inplace = True, axis =1)

In [ ]:
# save CBS directory
df.to_csv('CBS_directory.csv')